# Extract Lemmatization from JSON: Extended Parser
The code in this notebook will parse [ORACC](http://oracc.org) `JSON` files to extract lemmatization data for one or more projects. The resulting `csv` (Comma Separated Values) file is named `parsed.csv` and has two fields: a Text ID (e.g. `dcclt/Q000039`) and a string of lemmas in the format `lugal[king]N` (or `šarru[king]N` for Akkadian texts).

The output of the Extended Parser contains text IDs, line IDs, lemmas, and (potentially) other data. The first few code blocks are identical with the Basic Parser.

In [1]:
import pandas as pd
import zipfile
import json
import tqdm
import requests
import errno
import os

## 0 Create Directories, if Necessary
The two directories needed for this script are `jsonzip` and `output`. If they do not exist they are created, else: do nothing.

For the code, see [Stack Overflow](http://stackoverflow.com/questions/18973418/os-mkdirpath-returns-oserror-when-directory-does-not-exist).

In [2]:
directories = ['jsonzip', 'output']
for d in directories:
    try:
        os.mkdir(d)
    except OSError as exc:
        if exc.errno !=errno.EEXIST:
            raise
        pass

## 1.1 Input Project Names
Provide a list of one or more project names, separated by commas. Note that subprojects must be listed separately, they are not included in the main project. For instance:

`saao/saa01,saao/saa02,blms`

In [3]:
projects = input('Project(s): ').lower()

Project(s): riao


## 1.2 Split the List of Projects
Split the list of projects and create a list of project names.

In [4]:
p = projects.split(',')               # split at each comma and make a list called `p`
p = [x.strip() for x in p]        # strip spaces left and right of each entry in `p`

## 1.3 Download the ZIP files
For each project in the list download all the `json` files from `http://build-oracc.museum.upenn.edu/json/`. The file is called `PROJECT.zip` (for instance: `dcclt.zip`). For subprojects the file is called `PROJECT-SUBPROJECT.zip` (for instance `cams-gkab.zip`). 

For larger projects (such as [DCCLT](http://oracc.org/dcclt)) the `zip` file may be 25Mb or more. Downloading may take some time and it may be necessary to chunk the downloading process. The `iter_content()` function in the `requests` library takes care of that.

If you have downloaded the files by hand (and put them in the `jsonzip` directory) you may skip this cell and jump directly to section [2.1 The Parsejson() function](#head21).

In [5]:
CHUNK = 16 * 1024
for project in tqdm.tqdm(p):
    project = project.replace('/', '-')
    url = "http://build-oracc.museum.upenn.edu/json/" + project + ".zip"
    file = 'jsonzip/' + project + '.zip'
    r = requests.get(url)
    if r.status_code == 200:
        print("Downloading " + url + " saving as " + file)
        with open(file, 'wb') as f:
            for c in r.iter_content(chunk_size=CHUNK):
                f.write(c)
    else:
        print(url + " does not exist.")

  0%|                                                    | 0/1 [00:00<?, ?it/s]

100%|████████████████████████████████████████████| 1/1 [00:07<00:00,  7.46s/it]


## <a name="head21"></a>2.1 The `parsejson()` function
The `parsejson()` function is essentially identical with that function in `First_JSON_parser.ipynb`, but it fetches more data. The field `word_id` consists of three parts, namely a text ID, line ID, and word ID, in the format `Q000039.76.2` meaning: the second word in line 76 of text object `Q000039`. Note that `76` is not a line number strictly speaking but an object reference within the text object. Things like horizontal rulings, columns, and breaks also get object references. The `word_id` field allows us to put lines together in the proper order.

The field `label` is a human-legible label that refers a line or another part of the text; it may look like `o i 23` (obverse column 1 line 23) or `r v 23'` (reverse column 5 line 23 prime). The `label` field is used in online [ORACC](http://oracc.org) editions to indicate line numbers.

The fields `extent`, `scope`, and `state` give metatextual data about the condition of the object; they capture the number of broken lines or columns and similar information. 


In [6]:
def parsejson(text, parameters):
    for JSONobject in text["cdl"]:
        if "cdl" in JSONobject: 
            parsejson(JSONobject, parameters)
        if "label" in JSONobject:
            parameters["label"] = JSONobject['label']
        if "f" in JSONobject:
            lemma = JSONobject["f"]
            lemma["id_word"] = JSONobject["ref"]
            lemma['label'] = parameters["label"]
            lemma["id_text"] = parameters["id_text"]
            lemm_l.append(lemma)
        if "strict" in JSONobject and JSONobject["strict"] == "1":
            lemma = {key: JSONobject[key] for key in parameters["dollar_keys"]}
            lemma["id_word"] = JSONobject["ref"] + ".0"
            lemma["id_text"] = parameters["id_text"]
            lemm_l.append(lemma)
    return

## 2.2 Call the `parsejson()` function for every `JSON` file
The code in this cell will iterate through the list of projects entered above (1.1). For each project the `JSON` zip file is located in the directory `jsonzip`, named PROJECT.zip. The `zip` file contains a file that is called `corpus.json` that contains a full list of all the text IDs available in that corpus (P, Q, and X numbers) under the key `members`. This list is used to identify the files that contain the text data and that will be parsed. The `zip` file contains a directory `corpusjson` that holds the text files - each one is called `P######.json` (or `Q######.json` or `X######.json`).

Each of these files is extracted from the `zip` file and read with the command command `json.loads()`, which reads the json data and transforms it into a Python dictionary (a sequence of keys and values).

This dictionary, which is called `text` is now sent to the `parsejson()` function, with the text ID as second argument. The function adds lemmata to the `lemm_l` list.

In [7]:
lemm_l = []
parameters = {"label": None, "id_text": None, "dollar_keys" : ["extent", "scope", "state"]}
for project in p:
    file = "jsonzip/" + project.replace("/", "-") + ".zip"
    try:
        z = zipfile.ZipFile(file)       # create a Zipfile object
    except:
        print(file + " does not exist or is not a proper ZIP file")
        continue
    files = z.namelist()     # list of all the files in the ZIP
    files = [name for name in files if "corpusjson" in name and name[-5:] == '.json']                                                                                                  #that holds all the P, Q, and X numbers.
    for filename in files:                            #iterate over the file names
        id_text = project + filename[-13:-5] # id_text is, for instance, blms/P414332
        parameters["id_text"] = id_text
        try:
            text = z.read(filename).decode('utf-8')         #read and decode the json file of one particular text
            data_json = json.loads(text)                # make it into a json object (essentially a dictionary)
            parsejson(data_json, parameters)               # and send to the parsejson() function
        except:
            print(id_text + ' is not available or not complete')

## 3 Data Structuring
### 3.1 Transform the Data into a DataFrame
The word_l list is transformed into a Pandas dataframe for further manipulation.

For various reasons not all JSON files will have all data types that potentially exist in an [ORACC](http://oracc.org) signature. Only Sumerian words have a `base`, so if your data set has no Sumerian, this column will not exist in the DataFrame.  If a text has no breakage information in the form of `$ 1 line broken` (etc.) the fields `extent`, `scope`, and `state` do not exist. Where such fields are referenced in the code below (sections 2-4), the code may fail and you may need to take out some lines.

In [8]:
words = pd.DataFrame(lemm_l)
words = words.fillna('')   # replace NaN (Not a Number) with empty string
words

,cf,contrefs,delim,epos,extent,form,gdl,gw,headform,id_text,id_word,label,lang,norm,pos,scope,sense,state
0,Umannisuṭa,,,PN,,u₂-ma-an-ni-su-ṭa,"[{'v': 'u₂', 'delim': '-', 'id': 'Q005667.1.1....",servant of Šamši-Adad I,,riao/Q005667,Q005667.1.1,1,akk,Umannisuṭa,PN,,servant of Šamši-Adad I,
1,māru,,,N,,DUMU,"[{'role': 'logo', 'id': 'Q005667.2.1.0', 's': ...",son,,riao/Q005667,Q005667.2.1,2,akk,mār,N,,son,
2,Idin-x,,,PN,,i-din-{d}x-x,"[{'v': 'i', 'delim': '-', 'id': 'Q005667.2.2.0...","father of Umannisuṭa, servant of Šamši-Adad I",,riao/Q005667,Q005667.2.2,2,akk,Idin-x,PN,,"father of Umannisuṭa, servant of Šamši-Adad I",
3,wardu,,,N,,ARAD,"[{'role': 'logo', 'id': 'Q005667.3.1.0', 's': ...",slave,,riao/Q005667,Q005667.3.1,3,akk,warad,N,,servant,
4,Šamši-Adad_I,,,RN,,{d}UTU-ši-{d}IŠKUR,"[{'pos': 'pre', 'det': 'semantic', 'seq': [{'v...","son of Ila-kabkabu, vice-regent of Aššur",,riao/Q005667,Q005667.3.2,3,akk,Šamši-Adad,RN,,"son of Ila-kabkabu, vice-regent of Aššur",
5,ištu,,,PRP,,TA,"[{'role': 'logo', 'id': 'Q004608.2.1.0', 's': ...",from,,riao/Q004608,Q004608.2.1,85b,akk,ultu,PRP,,from,
6,Aliṣir,,,SN,,{URU}a-li-ṣir,"[{'pos': 'pre', 'det': 'semantic', 'seq': [{'r...",1,,riao/Q004608,Q004608.2.2,85b,akk,Aliṣir,SN,,1,
7,namāšu,,,V,,at-tu-muš,"[{'v': 'at', 'delim': '-', 'id': 'Q004608.2.3....",set (oneself) in motion,,riao/Q004608,Q004608.2.3,85b,akk,attumuš,V,,set (oneself) in motion,
8,ana,,,PRP,,a-na,"[{'v': 'a', 'delim': '-', 'id': 'Q004608.3.1.0...",to,,riao/Q004608,Q004608.3.1,86,akk,ana,PRP,,to,
9,tiāmtu,,,N,,tam-ti,"[{'v': 'tam', 'delim': '-', 'id': 'Q004608.3.2...",sea,,riao/Q004608,Q004608.3.2,86,akk,tâmti,N,,sea,


## 3.2 Remove Spaces and Commas from Guide Word and Sense
Spaces in Guide Word and Sense may cause trouble in computational methods in tokenization, or when saved in Comma Separated Values format. All spaces and commas are replaced by hyphens or nothing, respectively.

In [9]:
words['sense'] = [x.replace(' ', '-') for x in words['sense']]
words['sense'] = [x.replace(',', '') for x in words['sense']]
words['gw'] = [x.replace(' ', '-') for x in words['gw']]
words['gw'] = [x.replace(',', '') for x in words['gw']]

The columns in the resulting DataFrame correspond to the elements of a full [ORACC](http://oracc.org) signature, plus information about text, line, and word ids:
* base (Sumerian only)
* cf (Citation Form)
* cont (continuation of the base; Sumerian only)
* epos (Effective Part of Speech)
* form (transliteration, omitting all flags such as indication of breakage)
* frag (transliteration; including flags)
* gdl_utf8 (cuneiform)
* gw (Guide Word: main or first translation in standard dictionary)
* id_line (a line ID that begins with the six-digit P, Q, or X number of the text)
* id_text (six-digit P, Q, or X number)
* id_word (word ID that begins with the ID number of the line)
* label (traditional line number in the form o ii 2' (obverse column 2 line 2'), etc.)
* lang (language code, including sux, sux-x-emegir, sux-x-emesal, akk, akk-x-stdbab, etc)
* morph (Morphology; Sumerian only)
* norm (Normalization: Akkadian)
* norm0 (Normalization: Sumerian)
* pos (Part of Speech)
* sense (contextual meaning)
* sig (full ORACC signature)

Not all data elements (columns) are available for all words. Sumerian words never have a `norm`, Akkadian words do not have `norm0`, `base`, `cont`, or `morph`. Most data elements are only present when the word is lemmatized; only `lang`, `form`, `pos`, `id_word`, `id_line`, and `id_text` should always be there. An unlemmatized word has `pos` 'X' (for unknown). Broken words have `pos` 'u' (for 'unlemmatizable).

# 3.3 Manipulate for Analysis on Line level
For analyses that use a line as unit of analysis (e.g. lines in lexical texts as analyzed in the [Phylogenetics](https://github.com/ErinBecker/digital-humanities-phylogenetics) project) one may need to create lemmas and combine these into lines by using the `id_line` variable.

## 3.3.1 Create Lemmas and Adjust Bases
A lemma, [ORACC](http://oracc.org) style, combines Citation Form, GuideWord and POS into a unique reference to one particular lemma in a standard dictionary, as in `lugal[king]N` (Sumerian) or `šarru[king]N`. Usually, not all words in a text are lemmatized, because a word may be (partly) broken and/or unknown. Unlemmatized and unlemmatizable words will receive a place-holder lemmatization that consists of the transliteration of the word (instead of the Citation Form), with `NA` as GuideWord and `NA` as POS, as in `i-bu-x[NA]NA`. Note that `NA` is a string.

In [10]:
words["lemma"] = words.apply(lambda r: (r["cf"] + '[' + r["gw"] + ']' + r["pos"]) 
                            if r["cf"] != '' else r['form'] + '[NA]NA', axis=1)
words['lemma'] = [lemma if not lemma == '[NA]NA' else '' for lemma in words['lemma'] ] # kick out empty forms

## 3.3.2 Group by Line
In the `words` dataframe each word has a separate row. In order into change this to a line-by-line representation we use the Pandas `.groupby` function, using `id_text`, `id_line` and `label` fields as the sorting arguments. 

The field `id_line` is created by splitting `id_word` into three elements. The format of `id_word` is `IDtext.line.word`. The middle part, `id_line` is made into an integer so that it can be used to put the lines into their proper order (note that `id_line` is an abstract reference number that indicates the sequence of lines in a text object; `label` is a human-readable line number in the format `o ii 3`: obverse column 2, line 3). 

The fields that are aggregated are `lemma`, `extent`, `scope`, and `state`. The fields `extent`, `scope`, and `state` represent data on the number of broken lines. For instance, the notation `4 lines missing` in the [ORACC](http://oracc.org) edition will result in `extent = "4"`, `scope = "line"`, `state = "missing"` (note that the value of `extent` is a string and will be `"n"` if the number of missing lines or columns is unknown).

In [11]:
#words['id_line'] = [wordid[:wordid.rfind('.')+1] for wordid in words['id_word']]
words['id_line'] = [int(wordid.split('.')[1]) for wordid in words['id_word']]

In [12]:
lines = words.groupby([words['id_text'], words['id_line'], words['label']]).agg({
        'lemma': ' '.join,
        'extent': ''.join, 
        'scope': ''.join,
        'state': ''.join
    }).reset_index()
lines

,id_text,id_line,label,scope,extent,lemma,state
0,riao/Q004455,2,i 1,,,ana[to]PRP Ninurta[1]DN gašru[very-strong]AJ d...,
1,riao/Q004455,3,i 2,,,rēštû[first]AJ hāmimu[one-who-gathers-(to-ones...,
2,riao/Q004455,4,i 3,,,šamû[heaven]N u[and]CNJ erṣetu[earth]N pētû[on...,
3,riao/Q004455,5,i 4,,,munnarbu[fugitive]N ekdu[wild]AJ ša[of]DET lā[...,
4,riao/Q004455,6,i 5,,,ša[of]DET lā[not]MOD nakāru[be(come)-different...,
5,riao/Q004455,7,i 6,,,u[and]CNJ erṣetu[earth]N qātu[hand]N paqdu[ent...,
6,riao/Q004455,8,i 7,,,ezzu[furious]AJ lā[not]MOD pādû[forgiving]AJ š...,
7,riao/Q004455,9,i 8,,,nūru[light]N šamû[heaven]N erṣetu[earth]N mušp...,
8,riao/Q004455,10,i 9,,,šumu[name]N ilu[god]N mammāna[who(so)ever]XP l...,
9,riao/Q004455,11,i 10,,,šarru[king]N kiššatu[totality]N šarru[king]N l...,


## 3.3.3 Alternative: Lines in Normalized Transcription
This code essentially follows the pattern of the preceding. Before grouping words into lines, we need to create a dummy for words that have not been normalized, using the field `form`.

In [13]:
for norm, idx in enumerate(words["norm"]):
    if norm == "":
        words["norm"][idx] = words["form"][idx]

In [14]:
lines_norm = words.groupby([words['id_text'], words['id_line'], words['label']]).agg({
        'norm': ' '.join,
    }).reset_index()
lines_norm

,id_text,id_line,label,norm
0,riao/Q004455,2,i 1,ana Ninurta gešri dandanni ṣīru ašarēd ilāni q...
1,riao/Q004455,3,i 2,rēštû hāmim tuqmāte bukur Nudimmud qurād Igigi...
2,riao/Q004455,4,i 3,šamê u erṣeti pētû nagbē kābisi erṣeti rapašti...
3,riao/Q004455,5,i 4,munnarbu ekdu ša lā enû qibīt pîšu ašarēd kibr...
4,riao/Q004455,6,i 5,ša lā uttakkaru siqir šaptišu lēʾû rapšu apkal...
5,riao/Q004455,7,i 6,u erṣetim qātuššu paqdu šar tamhāri ālilu ša t...
6,riao/Q004455,8,i 7,ezzu lā pādû ša tībušu abūbu sāpin māt nakirē ...
7,riao/Q004455,9,i 8,nūr šamê erṣetim mušpardû qereb apsî muʾabbit ...
8,riao/Q004455,10,i 9,šumšu ili mamma lā innennû qaʾʾiš balāṭi ili r...
9,riao/Q004455,11,i 10,šar kiššati šar lā šanān šar kullat kibrāt erb...


## 4 Save Results in CSV file
The output file is called `parsed.csv` and is placed in the directory `output`. In most computers, `csv` files open automatically in Excel. This program does not deal well with `utf-8` encoding. If you intend to use the file in Excel, change `encoding ='utf-8'` to `encoding='utf-16'`. For usage in computational text analysis applications `utf-8` is usually preferred. 

(Alternatively, use the instructions [here](https://www.itg.ias.edu/content/how-import-csv-file-uses-utf-8-character-encoding-0) to import a `utf-8` file into Excel).

In [32]:
savefile =  'parsed.csv'
with open('output/' + savefile, 'w', encoding="utf-8") as w:
    lines.to_csv(w)

In [33]:
savefile =  'parsed_norm.csv'
with open('output/' + savefile, 'w', encoding="utf-8") as w:
    lines_norm.to_csv(w)

In [30]:
Q = lines.loc[lines["id_text"] == "riao/Q005947"]

In [31]:
Q

,id_text,id_line,label,scope,extent,lemma,state
11948,riao/Q005947,1,1,,,Tukulti-apil-Ešarra_I[Tiglath-pileser-I-king-o...,
11949,riao/Q005947,2,2,,,māru[son]N Aššur-reša-iši_I[king-of-Assyria]RN...,
11950,riao/Q005947,3,3,,,bītu[house]N Anu[1]DN u[and]CNJ Adad[1]DN bēlu...,
11951,riao/Q005947,4,4,,,epēšu[do]V kasāru[block]V,
